In [2]:
import polars as pl
import plotly.express as px
import pandas as pd
from polars_visualization import pl_config as config
from freqSevEda import FreqSevEDA
import plotly.io as pio
import json

# Data overview

- We have 1580 entries. Grouping by Loc ID and Año Póliza we saw that each policy is annual.

In [3]:
#path = r"C:\Users\abrah\OneDrive\Desktop\casoCAS\archivo\data\final-dataset.xlsx"
path2 = r"C:\Users\abrah\OneDrive\Desktop\casoCAS\archivo\data\final-dataset_V.5.5.xlsx"
df = pl.read_excel(path2, sheet_name="Sheet1")

with config():
    print(df)

shape: (1_580, 21)
| Loc ID | Año Póliza | Exposicion | Suma Asegurada | Prima   | Evento ID | Numero Siniestros | Mes  | Duración de la inundación (día… | Severidad de la inundación (es… | Precipitación (mm) | Incremento del Nivel del Río (… | Monto de siniestro | Latitud | Longitud | Ciudad        | Pais         | Codigo pais | Continente | Sub continente     | Continente intermedio |
|--------|------------|------------|----------------|---------|-----------|-------------------|------|---------------------------------|---------------------------------|--------------------|---------------------------------|--------------------|---------|----------|---------------|--------------|-------------|------------|--------------------|-----------------------|
| loc001 | 2,014      | 1          | 87,223,100     | 659,145 | null      | 0                 | null | null                            | null                            | null               | null                            | 0            

In [4]:
eda = FreqSevEDA(df, exposure="Exposicion", claimNb="Numero Siniestros", claimAmount="Monto de siniestro")
eda.interactive_graph()

Dropdown(description='Columns:', options=('-', 'Loc ID', 'Año Póliza', 'Exposicion', 'Suma Asegurada', 'Prima'…

Output()

In [10]:
eda.graphFreqSev("Ciudad").update_layout(template = "plotly_white").write_html(r"C:\Users\abrah\OneDrive\Desktop\casoCAS\docs\images\fregSevCity.html")

# Description by columns

- We have negative values for TIV (Is this a mistake?) how should we deal with those values? We have only one location with negative TIV loc068. In this case we will take absolute value of the TIV column, in that way we don't lose information.
- Information between years 2014 and 2023
- Is premium in USD dolars?
- Duration floods between 4 and 10 days
- Severity value between 2 and 5. Why we don't have value 1?
- Precipitation between 100 mm and 310 mm
- Raising of river between 1.2 M and 4.5 M

In [4]:
with config():
    print(df.describe())

#lets fix TIV values 

df = df.with_columns(pl.col("Suma Asegurada").abs())


shape: (9, 22)
| statistic  | Loc ID | Año Póliza | Exposicion | Suma Asegurada    | Prima          | Evento ID | Numero Siniestros | Mes       | Duración de la inundación (día… | Severidad de la inundación (es… | Precipitación (mm) | Incremento del Nivel del Río (… | Monto de siniestro | Latitud   | Longitud  | Ciudad | Pais      | Codigo pais | Continente | Sub continente            | Continente intermedio |
|------------|--------|------------|------------|-------------------|----------------|-----------|-------------------|-----------|---------------------------------|---------------------------------|--------------------|---------------------------------|--------------------|-----------|-----------|--------|-----------|-------------|------------|---------------------------|-----------------------|
| count      | 1580   | 1,580.000  | 1,580.000  | 1,580.000         | 1,580.000      | 188.000   | 1,580.000         | 188.000   | 188.000                         | 188.000               

# Information against Año Póliza

- Precipitación Promedio per years tends to increase

In [5]:
w = df.filter()

TypeError: at least one predicate or constraint must be provided

In [6]:
summary = (df.group_by(["Pais"]).agg(pl.col("Exposicion").sum().alias("Exposicion"),
                              pl.col("Suma Asegurada").sum(),
                              pl.col("Prima").sum().alias("Prima Ganada"),
                              pl.col("Evento ID").drop_nulls().n_unique().alias("Número Eventos"),
                              pl.col("Numero Siniestros").sum(),
                              pl.col("Duración de la inundación (días)").mean().alias("Duración Promedio"),
                              pl.col("Severidad de la inundación (escala 1-5)").mean().alias("Magnitud Inundaciones Promedio"),
                              pl.col("Precipitación (mm)").mean().alias("Precipitación Promedio"),
                              pl.col("Incremento del Nivel del Río (m)").mean().alias("Incremento Promedio"),
                              pl.col("Monto de siniestro").sum().alias("Incurrido"))
                         .with_columns((pl.col("Incurrido")/pl.col("Prima Ganada")).alias("Indice de Siniestralidad"),
                                       (pl.col("Prima Ganada")/pl.col("Suma Asegurada")*1000).alias("Tasa por Mil"),
                                       (pl.col("Incurrido")/pl.col("Suma Asegurada")).alias("TPR"),
                                       (pl.col("Numero Siniestros")/pl.col("Exposicion")).alias("Frecuencia"),
                                       (pl.col("Incurrido")/pl.col("Numero Siniestros")).alias("Severidad"))
                         .with_columns((pl.col("Frecuencia")*pl.col("Severidad")).alias("Prima Pura"))
                         .with_columns((pl.col("Prima Pura")/pl.col("Prima Ganada")).alias("proportion"))
                         .with_columns((pl.col("Suma Asegurada")/pl.col("Exposicion")).alias("Suma Asegurada Promedio"))
                         .sort(by = "Indice de Siniestralidad", descending=False))

summary

Pais,Exposicion,Suma Asegurada,Prima Ganada,Número Eventos,Numero Siniestros,Duración Promedio,Magnitud Inundaciones Promedio,Precipitación Promedio,Incremento Promedio,Incurrido,Indice de Siniestralidad,Tasa por Mil,TPR,Frecuencia,Severidad,Prima Pura,proportion,Suma Asegurada Promedio
str,i64,i64,i64,u32,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""Brazil""",70,10707066620,66963252,2,7,4.571429,2.571429,151.428571,1.871429,1400000,0.020907,6.254117,0.000131,0.1,200000.0,20000.0,0.000299,1.5296e8
"""New Zealand""",80,8436502580,80166073,2,8,5.0,3.0,155.0,1.8,1680000,0.020956,9.502287,0.000199,0.1,210000.0,21000.0,0.000262,1.0546e8
"""Italy""",60,15946810870,138648915,2,6,6.0,3.333333,196.666667,2.4,3400000,0.024522,8.69446,0.000213,0.1,566666.666667,56666.666667,0.000409,2.6578e8
"""China""",70,33536257010,287561631,2,7,6.571429,4.0,230.0,3.371429,7490000,0.026047,8.574649,0.000223,0.1,1.07e6,107000.0,0.000372,4.7909e8
"""South Africa""",80,6805602900,58991325,2,8,7.0,4.0,170.0,2.3,1660000,0.02814,8.668053,0.000244,0.1,207500.0,20750.0,0.000352,8.5070e7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""United Kingdom""",30,12867347490,40476091,1,3,10.0,5.0,200.0,3.5,3010000,0.074365,3.145644,0.000234,0.1,1.0033e6,100333.333333,0.002479,4.28911583e8
"""Netherlands""",30,3295486380,16382808,1,3,8.0,4.0,240.0,3.1,1360000,0.083014,4.971287,0.000413,0.1,453333.333333,45333.333333,0.002767,1.09849546e8
"""Peru""",30,2913588680,20344473,3,5,4.6,2.4,134.0,1.42,1690000,0.083069,6.982617,0.00058,0.166667,338000.0,56333.333333,0.002769,9.7120e7


# Distribution by country

In [41]:
json_path = r"C:\Users\abrah\OneDrive\Desktop\casoCAS\archivo\data\countries.geojson"
# Open and read the JSON file
with open(json_path) as file:
    countries = json.load(file)  # Parse the JSON data into a Python dictionary/list

In [42]:
pio.renderers
pio.renderers.default = "notebook_connected"

In [ ]:
fig2 = px.choropleth_map(summary,
                        geojson=countries,
                        color="Suma Asegurada Promedio",
                        locations='Codigo pais',
                        featureidkey="properties.ISO_A3",
                        #center={"lat": 0, "lon": 0},
                        #map_style=#carto-voyager",
                        template="plotly_white",
                        color_continuous_scale = "YlOrRd", #PuRd
                        hover_name = "Pais",
                        zoom=0)


fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, height = 300)
fig2.show()
fig2.write_html(r"C:\Users\abrah\OneDrive\Desktop\casoCAS\docs\images\sumaAseguradaPromedio2.html")

In [39]:
fig2 = px.choropleth_map(summary,
                        geojson=countries,
                        color="Suma Asegurada",
                        locations='Codigo pais',
                        featureidkey="properties.ISO_A3",
                        #center={"lat": 0, "lon": 0},
                        #map_style=#carto-voyager",
                        template="plotly_white",
                        color_continuous_scale = "YlOrRd", #PuRd
                        hover_name = "Pais",
                        zoom=0)

fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, height = 300)
fig2.write_html(r"C:\Users\abrah\OneDrive\Desktop\casoCAS\docs\images\SumaAsegurada2023.html")

# Heat map indice de siniestralidad

In [7]:
f = summary.filter(pl.col("Numero Siniestros")!=0)

In [8]:
fig = px.density_heatmap(summary, x = "Pais", y="Magnitud Inundaciones Promedio", text_auto = False, z = "Indice de Siniestralidad",color_continuous_scale = "YlOrRd", template = "plotly_white")

fig.update_layout( coloraxis_colorbar_title_text = 'Índice Siniestralidad')

#fig.write_html(r"C:\Users\abrah\OneDrive\Desktop\casoCAS\docs\images\SiniestralidadVsSeveridad.html")
